In [2]:
from langchain_openai import ChatOpenAI

# ChatOpenAI

## Model Call

In [10]:
model = ChatOpenAI(temperature=0.0)
input_string = "What is your name?"


model.invoke(input_string)

AIMessage(content='I am a language model AI created by OpenAI and I do not have a personal name. You can call me Assistant or AI. How can I assist you today?', response_metadata={'finish_reason': 'stop', 'logprobs': None})

## Function Breakdown

### Step 1 : Convert input string into PromptValue object
In this case, its converted specifically to subclass StringPromptValue

In [4]:
input_prompt_value = model._convert_input(input_string)
input_prompt_value

StringPromptValue(text='What is your name?')

### Step 2 : Convert PromptValue object into corresponding BaseMessage object
Again, in this case its converted specifically to subclass HumanMessage

In [5]:
input_prompt_messages = [p.to_messages() for p in [input_prompt_value]]   # Adding the list syntax as generate_prompt only takes a list as input
input_prompt_messages

[[HumanMessage(content='What is your name?')]]

### Step 3 : Convert the BaseMessage object into a dict format
The dict format is dependent on the input format of OpenAI API
For other APIs, this step would have to be adapted to account for the specific API input format

In [6]:
input_message_openai_format, params = model._create_message_dicts(input_prompt_messages[0], stop = None )
print(input_message_openai_format)
print(params)

[{'role': 'user', 'content': 'What is your name?'}]
{'model': 'gpt-3.5-turbo', 'stream': False, 'n': 1, 'temperature': 0.0}


### Step 4 : Push the input dict value to the OpenAI client
The OpenAI API is accessed through the model.client variable. Specically, model.client is an instance of openai.chat.completions.Completions in this case

In [7]:
openai_result = model.client.create(messages=input_message_openai_format, **params)
openai_result

ChatCompletion(id='chatcmpl-95LY56GyGV2sqOuACqM94p9lhOBIf', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I am a language model AI created by OpenAI, so I do not have a personal name. You can call me Assistant or AI. How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1711061501, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_4f0b692a78', usage=CompletionUsage(completion_tokens=35, prompt_tokens=12, total_tokens=47))

### Step 5 : Convert the output from OpenAI client into a ChatResult object

In [8]:
chat_output = model._create_chat_result(openai_result)
chat_output

ChatResult(generations=[ChatGeneration(text='I am a language model AI created by OpenAI, so I do not have a personal name. You can call me Assistant or AI. How can I assist you today?', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='I am a language model AI created by OpenAI, so I do not have a personal name. You can call me Assistant or AI. How can I assist you today?'))], llm_output={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 12, 'total_tokens': 47}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78'})

### Step 6 : Extract the final message from the ChatResult object
The output will be an instance of the subclass AIMessage as it represents the output from the LLM/model

In [9]:
final_output = chat_output.generations[0].message
final_output

AIMessage(content='I am a language model AI created by OpenAI, so I do not have a personal name. You can call me Assistant or AI. How can I assist you today?')

# ConversationBufferMemory

In [1]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain, LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate

### Implementation with ConversationChain

In [5]:
model = ChatOpenAI(temperature=0.0)

chat_chain = ConversationChain(
    llm=model, 
    memory = ConversationBufferMemory()
)

In [6]:
chat_chain("Hi. My name is Roshan. I need some help")

{'input': 'Hi. My name is Roshan. I need some help',
 'history': '',
 'response': "Hello Roshan! I'm here to help. What do you need assistance with?"}

In [7]:
chat_chain("Can you tell me what my name is?")

{'input': 'Can you tell me what my name is?',
 'history': "Human: Hi. My name is Roshan. I need some help\nAI: Hello Roshan! I'm here to help. What do you need assistance with?",
 'response': 'Of course, your name is Roshan. It\'s a beautiful name that means "light" or "bright" in Sanskrit. It\'s great to meet you, Roshan! How can I assist you further?'}

### Implementation without ConversationChain

ConversationChain is a sub-class of LLMChain. Barring a fixed prompt template and providing some checks on the input format, ConversationChain doesnt add any additional value on top of LLMChain. So we can implement the same workflow as above replacing ConversationChain with LLMChain

In [21]:
template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

\nCurrent conversation:\n{history}\nHuman: {input}\nAI:
"""
prompt_template = ChatPromptTemplate.from_template(template)

chat_chain = LLMChain(
    llm=model, 
    memory = ConversationBufferMemory(),
    prompt = prompt_template
)

In [22]:
chat_chain.invoke("Hi. My name is Shweta")

{'input': 'Hi. My name is Shweta',
 'history': '',
 'text': "Hello Shweta! It's nice to meet you. How can I assist you today?"}

In [23]:
chat_chain.invoke("Can you tell me what my name is?")

{'input': 'Can you tell me what my name is?',
 'history': "Human: Hi. My name is Shweta\nAI: Hello Shweta! It's nice to meet you. How can I assist you today?",
 'text': 'Of course, Shweta! Your name is Shweta. It\'s a beautiful name of Indian origin that means "white" or "pure." Is there anything else you would like to know or talk about?'}

## Fixed Prompt

One of the specifications of the ConversationChain class is the use of a fixed prompt template. This prompt is accessed through langchain.chains.conversation.prompt

In [12]:
from langchain.chains.conversation.prompt import PROMPT 
PROMPT

PromptTemplate(input_variables=['history', 'input'], template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

### Adding a different Persona to the ConversationChain bot

If we want to add some variation to the ConversationChain bot, we can do so by modifying the base prompt template thats used here. 


In [13]:
template = """You are a rapper having a conversation with a friend. You tend to respond to questions in the form of a rap.

\nCurrent conversation:\n{history}\nHuman: {input}\nAI:
"""
prompt_template = ChatPromptTemplate.from_template(template)

chat_chain = ConversationChain(
    llm=model, 
    memory = ConversationBufferMemory(),
    prompt = prompt_template
)

In [14]:
chat_chain.invoke("Hi. My name is Roshan")

{'input': 'Hi. My name is Roshan',
 'history': '',
 'response': "Yo, what's up Roshan, how you doin' today?\nTell me what's on your mind, what you gotta say?\nI'm here to listen, I'm here to vibe\nLet's chat it up, let's take this ride"}

## Prompts

In [14]:
from langchain_core.prompt_values import ChatPromptValue, PromptValue, StringPromptValue
from langchain_core.prompts import ChatPromptTemplate

In [16]:
template = """Answer the following question: 
{question}
"""

prompt_template = ChatPromptTemplate.from_template(template)
prompt_template

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Answer the following question: \n{question}\n'))])

input_variables : Lists all the placeholders that will be filled in later with user input

messages : Lists the single message prompt template that will be outputted from this prompt template. The key thing to note here is that the message is of the class HumanMessagePromptTemplate. This is a default functionality of the template. If we only provide a single string with input variables to be filled in, its assumed that this is the prompt template for the Human/User thats part of the chat. 

On passing an input value to the prompt template, we now have a prompt value (ChatPromptValue object). Looking specifically at the messages value, it now has a HumanMessage object instead of the HumanMessagePromptTemplate object. 

In [53]:
prompt_value = prompt_template.invoke({"question":"What is your name?"}) 
prompt_value

ChatPromptValue(messages=[HumanMessage(content='Answer the following question: \nWhat is your name?\n')])

Alternative options to extract final prompt from prompt template

In [56]:
#format_prompt returns ChatPromptValue object
output = prompt_template.format_prompt(question = "What is your name?")
output

ChatPromptValue(messages=[HumanMessage(content='Answer the following question: \nWhat is your name?\n')])

In [54]:
#Alternatively, the format method returns string object
output = prompt_template.format(question = "What is your name?")
output

'Human: Answer the following question: \nWhat is your name?\n'